In [1]:
# Author: Vatsal Sanjay
# vatsalsanjay@gmail.com
# Physics of Fluids
# Last updated: Dec 24, 2024

import numpy as np
import os
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.ticker import StrMethodFormatter
import multiprocessing as mp
from functools import partial
import argparse  # Add at top with other imports
import sys
import matplotlib.colors as mcolors
np.set_printoptions(threshold=sys.maxsize)
GridsPerR = 128
lw = 2

This is used to reproduce simulation results

Plot analytic and simulation results separately

In [2]:
wavelength=1.07
period=1.14
def u_analytic(x,y,t):   #Here u is u/sqrt(g\lambda)
    ux_analytic=np.zeros(1)
    uy_analytic=np.zeros(1)
    u_analytic=np.zeros(1)
    u_analytic=abs((0.1*np.pi*2/wavelength) * np.exp(2*np.pi*y/wavelength)*np.sin(2*np.pi*t/period))
    ux_analytic=(0.1*np.pi*2/wavelength) * np.exp(2*np.pi*y/wavelength)*np.sin(2*np.pi*x/wavelength)*np.sin(2*np.pi*t/period)
    uy_analytic=(0.1*np.pi*2/wavelength) * np.exp(2*np.pi*y/wavelength)*np.cos(2*np.pi*x/wavelength)*np.sin(2*np.pi*(t-0.57)/period)
    return [u_analytic, ux_analytic, uy_analytic]

def comparison(t):
    data = np.load(f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/{int(t):08d}_data.npz")
    t=t/1000.
    X =data['X']
    Y = data['Y']
    vel_rot   = data['vel_rot']
    omega_rot = data['omega_rot']
    f_rot     = data['f_rot']
    ux_rot    = data['ux_rot']
    uy_rot    = data['uy_rot']

    zmin=-1.0
    zmax=1.0
    rmin=-1.0
    rmax=1.0
    # ----------------------------------------------------------
    x_extent = [zmin, zmax]
    y_extent = [rmin, rmax]

    # For imshow, extent is [x_min, x_max, y_min, y_max].
    extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
    #Initilize field matrices
    dimension=np.shape(X)[0]
    surface=np.zeros((dimension,2))
    percentage=np.zeros((dimension,dimension))
    f_gradient=np.zeros(dimension)
    u_th_field=np.zeros((dimension,dimension))
    ux_th_field=np.zeros((dimension,dimension))
    uy_th_field=np.zeros((dimension,dimension))
    [X_quiver, Y_quiver, ux_rot_quiver, uy_rot_quiver, ux_th_field_quiver, uy_th_field_quiver]=[np.zeros((30,30)), np.zeros((30,30)), np.zeros((30,30)), np.zeros((30,30)), np.zeros((30,30)), np.zeros((30,30))]
    for i in range(0,dimension-1):
        for j in range(0,dimension):
            #Get the coordinates of points where the volume fraction gradient is highest.
            if abs(f_rot[i+1,j]-f_rot[i,j]) > f_gradient[j]:  
                f_gradient[j] = abs(f_rot[i+1,j]-f_rot[i,j])
                surface[j,0] = X[i,j]
                surface[j,1] = Y[i,j]


    for i in range(dimension):
        for j in range(dimension):
            x=X[i,j]
            y=Y[i,j]
            u=vel_rot[i,j]
            [u_th,ux_th,uy_th]=u_analytic(x,y,t)
            u_th_field[i,j]=u_th
            ux_th_field[i,j]=ux_th
            uy_th_field[i,j]=uy_th
           # percentage[i,j]=(u-u_th)/u_th
            if y > surface_analytic(x,t):
                #percentage[i,j]=0
                u_th_field[i,j]=0
                ux_th_field[i,j]=0
                uy_th_field[i,j]=0
            if i%40 == 0 and j%40 == 0:
                X_quiver[int(i/40),int(j/40)] = X[i,j]
                Y_quiver[int(i/40),int(j/40)] = Y[i,j]
                ux_rot_quiver[int(i/40),int(j/40)] = ux_rot[i,j]
                uy_rot_quiver[int(i/40),int(j/40)] = uy_rot[i,j]
                ux_th_field_quiver[int(i/40),int(j/40)] = ux_th_field[i,j]
                uy_th_field_quiver[int(i/40),int(j/40)] = uy_th_field[i,j]
   
    return [percentage,surface,X,dimension,u_th_field,vel_rot,ux_th_field,ux_rot,uy_th_field,uy_rot,X,Y,X_quiver, Y_quiver, ux_rot_quiver, uy_rot_quiver, ux_th_field_quiver, uy_th_field_quiver]

def surface_analytic(x,t):
    eta=0
    eta=0.05*np.cos(2*np.pi * x/wavelength) * np.cos(2*np.pi/period *t)
    return eta

for t in range(0,2000,10):
    for plot_range in np.array((0.2,0.5)):
        name = f"/home/gridsan/yhe/postprocess/dataFiles_Order_1/comparison/plot_range={int(plot_range*10)}/{int(t):08d}.png"
        [percentage,surface,X,dimension,u_th_field,vel_rot,ux_th_field,ux_rot,uy_th_field,uy_rot,X,Y,X_quiver, Y_quiver, ux_rot_quiver, uy_rot_quiver, ux_th_field_quiver, uy_th_field_quiver] = comparison(t)
        AxesLabel, TickLabel = 50, 20

        zmin=-1.0
        zmax=1.0
        rmin=-1.0
        rmax=1.0
        # ----------------------------------------------------------
        x_extent = [zmin, zmax]
        y_extent = [rmin, rmax]

        # For imshow, extent is [x_min, x_max, y_min, y_max].
        extent_vel   = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]
        extent_omega = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]

        fig, axes = plt.subplots(3, 2, figsize=(19.20, 10.80))
        
        #ax1: total velocity magnitude for simulation. ax2: total velocity magnitude from analytic.
        #ax3: x-velocity, simulation.                  ax4: x_velocity, analytic.
        #ax5: y-velocity, simulation.                  ax6: y-velocity, analytic.
        
        ax1 = axes[0, 0]
        ax2 = axes[0, 1]
        ax3 = axes[1, 0]
        ax4 = axes[1, 1]
        ax5 = axes[2, 0]
        ax6 = axes[2, 1]

        # Draw the rotated interface in green, domain boundaries, etc.
        for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
            # Gray dashed lines now get swapped similarly if you want
            # them at z=0 or r=0.  If you simply want a bounding box,
            # note that x->z, y->r:
            ax.plot([-1.0, 1.0], [0, 0], '--', color='grey', linewidth=2)  # "horizontal" axis is z
            ax.plot([0, 0], [-1.0, 1.0], '-.', color='grey', linewidth=2)  # "vertical" axis is r

            # Domain box:
            ax.plot([-1.0, 1.0], [-1.0, -1.0], '-', color='black', linewidth=2)
            ax.plot([-1.0, 1.0], [1.0, 1.0], '-', color='black', linewidth=2)
            ax.plot([-1.0, -1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
            ax.plot([1.0, 1.0], [-1.0, 1.0], '-', color='black', linewidth=2)
            surface_plot = [np.column_stack([surface[:,0], surface[:,1]]) for j in range(0,1)]
            surface_analytic_plot = [np.column_stack([X[512,:], surface_analytic(X[512,:],t/1000.)]) for j in range(0,1)]
            if ax == ax1 or ax == ax3 or ax == ax5:  #ax1,3,5 should have simulation surface.

                line_segments = LineCollection(surface_plot, linewidths=4, colors='green')
                ax.add_collection(line_segments)
            else:                                    #ax2,4,6 should have analytic surface.
                line_segments_analytic = LineCollection(surface_analytic_plot, linewidths=4, colors='red')
                ax.add_collection(line_segments_analytic)

        # ----------------------------------------------------------
        # 5) Now show imshow with the rotated arrays and extents:
        # ----------------------------------------------------------
        #Total velocity magnitude field, simulation
        plot_level=15
        cntrl1 = ax1.imshow(
            vel_rot, 
            cmap=plt.get_cmap('Blues', plot_level),
            interpolation='bilinear', 
            origin='lower', 
            extent=extent_vel,
            vmin=0.0,
            vmax=plot_range
        )
        
        #Total velocity magnitude field, analytic
        cntrl2 = ax2.imshow(
        u_th_field, 
        cmap=plt.get_cmap('Blues', plot_level),
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=0.0,
        vmax=plot_range
        )
        
        #Total x-velocity field, simulation
        cntrl3 = ax3.imshow(
        ux_rot, 
        cmap=plt.get_cmap('coolwarm', plot_level),
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=-plot_range,
        vmax=plot_range
        )
        
        #Total x-velocity field, analytic.
        cntrl4 = ax4.imshow(
        ux_th_field, 
        cmap=plt.get_cmap('coolwarm', plot_level),
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=-plot_range,
        vmax=plot_range
        )
        
        #Total y-velocity field, simulation
        cntrl5 = ax5.imshow(
        uy_rot, 
        cmap=plt.get_cmap('coolwarm', plot_level),
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=-plot_range,
        vmax=plot_range
        )
        
        #Total y-velocity field, analytic
        cntrl6 = ax6.imshow(
        uy_th_field, 
        cmap=plt.get_cmap('coolwarm', plot_level),
        interpolation='bilinear', 
        origin='lower', 
        extent=extent_omega,
        vmin=-plot_range,
        vmax=plot_range
        )

        
        
        # Equal aspect ensures squares in the new orientation
        for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
            ax.set_aspect('equal')
            ax.set_xlim(zmin, zmax)  # x range
            ax.set_ylim(rmin, rmax)  # y range
            #Plot quivers
            if ax == ax1 or ax == ax3 or ax == ax5:
                ax.quiver(X_quiver, Y_quiver, ux_rot_quiver, uy_rot_quiver, 
                          scale=5,          # Adjust arrow length
                          scale_units='xy', # Interpret arrow length in x-y data units
                          width=0.005,      # Thicker arrows
                          color='black')
            else:
                ax.quiver(X_quiver, Y_quiver, ux_th_field_quiver, uy_th_field_quiver,
                          scale=5,          # Adjust arrow length
                          scale_units='xy', # Interpret arrow length in x-y data units
                          width=0.005,      # Thicker arrows
                          color='black')

        # Titles and labels that match the new orientation
        ax1.set_title(fr'$t\sqrt{{g/\lambda}} = {t/1000:.4f}$ (Velocity)', fontsize=TickLabel)
        # ax1.set_xlabel('$X$', fontsize=AxesLabel)
        # ax1.set_ylabel('$r$', fontsize=AxesLabel)

        ax2.set_title(r'analytic', fontsize=TickLabel)
        # ax2.set_xlabel('$z$', fontsize=AxesLabel)
        # ax2.set_ylabel('$r$', fontsize=AxesLabel)
        # Colorbars: place them below each subplot, for instance
        

        fig.subplots_adjust(bottom=0.2, wspace=0.3)  # more spacing for colorbars
        cbar_ax1 = fig.add_axes([0.125, 0.68, 0.35, 0.01])   # x,y,width,height in figure coords
        c1 = plt.colorbar(cntrl1, cax=cbar_ax1, orientation='horizontal')
        c1.ax.tick_params(labelsize=TickLabel)
  

        cbar_ax2 = fig.add_axes([0.57, 0.68, 0.35, 0.01])
        c2 = plt.colorbar(cntrl2, cax=cbar_ax2, orientation='horizontal')
        c2.ax.tick_params(labelsize=TickLabel)
    
        
        cbar_ax3 = fig.add_axes([0.125, 0.43, 0.35, 0.01])
        c3 = plt.colorbar(cntrl3, cax=cbar_ax3, orientation='horizontal')
        c3.ax.tick_params(labelsize=TickLabel)

        
        cbar_ax4 = fig.add_axes([0.57, 0.43, 0.35, 0.01])
        c4 = plt.colorbar(cntrl4, cax=cbar_ax4, orientation='horizontal')
        c4.ax.tick_params(labelsize=TickLabel)
   
        
        cbar_ax5 = fig.add_axes([0.125, 0.1, 0.35, 0.01])
        c5 = plt.colorbar(cntrl5, cax=cbar_ax5, orientation='horizontal')
        c5.ax.tick_params(labelsize=TickLabel)
    
        
        cbar_ax6 = fig.add_axes([0.57, 0.1, 0.35, 0.01])
        c6 = plt.colorbar(cntrl6, cax=cbar_ax6, orientation='horizontal')
        c6.ax.tick_params(labelsize=TickLabel)
   

        for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
            ax.axis('off')


        plt.savefig(name, bbox_inches="tight",dpi=300)
        plt.close()



In [17]:
X[0,512]

0.000976562